# SF3D API Server - JupyterHub

This notebook runs a FastAPI server on the school GPU to generate 3D meshes using Stable Fast 3D.

**Setup Steps:**
1. Upload this notebook to JupyterHub
2. Install dependencies (first cell)
3. Run the server cell
4. Keep this notebook running
5. Use from your Mac: `python tests/sf3d_api_client.py <image_path>`

**Server will be available at:** `http://itp-ml.itp.tsoa.nyu.edu:<PORT>/`

**Note**: You need to be on NYU network/VPN to access this server.

## 1. Install Dependencies

In [ ]:
# Install required packages
!pip install -q fastapi uvicorn[standard] python-multipart
!pip install -q sf3d  # Install SF3D from Stability AI
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu118  # Match CUDA 11.8
!pip install -q pillow numpy trimesh

print("✅ Dependencies installed!")

## 2. Import Libraries and Initialize Model

In [ ]:
import io
import os
import time
from pathlib import Path
from typing import Optional

import torch
import numpy as np
from PIL import Image
from fastapi import FastAPI, File, UploadFile, Form, HTTPException
from fastapi.responses import FileResponse, JSONResponse
from fastapi.middleware.cors import CORSMiddleware
import uvicorn

# Try to import SF3D
try:
    from sf3d.system import SF3D
    print("✅ SF3D imported successfully")
except ImportError:
    print("❌ SF3D not found. Trying alternative import...")
    # Fallback: If sf3d package not available, we'll use transformers pipeline
    from transformers import pipeline
    print("✅ Using transformers pipeline as fallback")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 3. Load SF3D Model

In [ ]:
# Configure device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Load model
print("Loading SF3D model...")
start_time = time.time()

try:
    # Try official SF3D implementation
    model = SF3D.from_pretrained(
        "stabilityai/stable-fast-3d",
        config_name="config.yaml",
        weight_name="model.safetensors",
    )
    model = model.to(device)
    model.eval()
    print(f"✅ SF3D model loaded in {time.time() - start_time:.1f}s")
except Exception as e:
    print(f"Failed to load SF3D: {e}")
    print("Trying alternative loading method...")
    # Alternative: Use transformers pipeline
    model = pipeline("image-to-3d", model="stabilityai/stable-fast-3d", device=0 if device == "cuda" else -1)
    print(f"✅ Model loaded via transformers in {time.time() - start_time:.1f}s")

# Create output directory
output_dir = Path("/tmp/sf3d_outputs")
output_dir.mkdir(exist_ok=True)
print(f"Output directory: {output_dir}")

## 4. Define API Endpoints

In [ ]:
# Initialize FastAPI
app = FastAPI(
    title="SF3D API Server",
    description="Generate 3D meshes from images using Stable Fast 3D",
    version="1.0.0"
)

# Enable CORS
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

@app.get("/")
async def root():
    return {
        "message": "SF3D API Server",
        "status": "running",
        "device": device,
        "endpoints": {
            "/generate": "POST - Generate 3D mesh from image",
            "/health": "GET - Health check"
        }
    }

@app.get("/health")
async def health():
    return {
        "status": "healthy",
        "device": device,
        "cuda_available": torch.cuda.is_available(),
        "model_loaded": model is not None
    }

@app.post("/generate")
async def generate_mesh(
    file: UploadFile = File(...),
    texture_resolution: int = Form(1024),
    remesh_option: str = Form("none"),
    foreground_ratio: float = Form(0.85)
):
    """
    Generate 3D mesh from uploaded image.
    
    Parameters:
    - file: Image file (PNG, JPG)
    - texture_resolution: 512, 1024, or 2048 (default: 1024)
    - remesh_option: 'none', 'triangle', or 'quad' (default: 'none')
    - foreground_ratio: 0.5-1.0 (default: 0.85)
    
    Returns:
    - GLB file
    """
    try:
        # Read and validate image
        image_data = await file.read()
        image = Image.open(io.BytesIO(image_data))
        
        # Convert to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        print(f"Received image: {image.size}, mode: {image.mode}")
        print(f"Settings: texture={texture_resolution}, remesh={remesh_option}, fg_ratio={foreground_ratio}")
        
        # Generate mesh
        start_time = time.time()
        
        with torch.no_grad():
            # SF3D inference
            if hasattr(model, 'run'):
                # Official SF3D method
                output = model.run(
                    image,
                    texture_resolution=texture_resolution,
                    remesh=remesh_option if remesh_option != 'none' else None,
                    foreground_ratio=foreground_ratio
                )
            else:
                # Transformers pipeline method
                output = model(image)
        
        generation_time = time.time() - start_time
        print(f"Generation completed in {generation_time:.2f}s")
        
        # Save mesh to temporary file
        timestamp = int(time.time() * 1000)
        output_path = output_dir / f"mesh_{timestamp}.glb"
        
        # Export mesh (SF3D typically outputs dict with 'mesh' key)
        if isinstance(output, dict):
            mesh = output.get('mesh', output)
        else:
            mesh = output
        
        # Save as GLB
        if hasattr(mesh, 'export'):
            mesh.export(str(output_path))
        else:
            # If mesh is already bytes
            with open(output_path, 'wb') as f:
                f.write(mesh)
        
        file_size = output_path.stat().st_size
        print(f"Saved mesh: {output_path} ({file_size / 1024:.1f} KB)")
        
        # Return file
        return FileResponse(
            path=output_path,
            media_type="model/gltf-binary",
            filename=f"mesh_{timestamp}.glb",
            headers={
                "X-Generation-Time": str(generation_time),
                "X-File-Size": str(file_size)
            }
        )
    
    except Exception as e:
        print(f"Error generating mesh: {e}")
        import traceback
        traceback.print_exc()
        raise HTTPException(status_code=500, detail=str(e))

print("✅ API endpoints defined")

## 5. Start Server

**IMPORTANT**: 
- Change `PORT` to an available port (8000-9000 range)
- This cell will run indefinitely - keep the notebook open
- Server URL will be: `http://itp-ml.itp.tsoa.nyu.edu:<PORT>/`

In [ ]:
# Configure server
PORT = 8765  # Change this if port is already in use
HOST = "0.0.0.0"  # Listen on all interfaces

print("="*70)
print("🚀 Starting SF3D API Server")
print("="*70)
print(f"Server URL: http://itp-ml.itp.tsoa.nyu.edu:{PORT}/")
print(f"Device: {device}")
print(f"")
print("Available endpoints:")
print(f"  GET  /           - API info")
print(f"  GET  /health     - Health check")
print(f"  POST /generate   - Generate 3D mesh")
print(f"")
print("To test from your Mac:")
print(f"  python tests/sf3d_api_client.py <image_path> --server http://itp-ml.itp.tsoa.nyu.edu:{PORT}")
print("="*70)
print("")
print("⚠️  KEEP THIS CELL RUNNING - Press ■ to stop server")
print("")

# Run server
uvicorn.run(app, host=HOST, port=PORT)